In [30]:
import numpy as np
import pandas as pd
import requests
from census import Census
from sqlalchemy import create_engine

api_key = "85ac64b6b5a9c0901b00329d1ef41f0c53ccfc98"

In [31]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
def project_data(api, year):
    
    c = Census(api_key, year=year)
    
    census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B17012_002E",
                          "B19301_001E",
                          "B17001_002E",
                          "B25064_001E",
                          "B23025_005E",
                          "B02001_003E",
                          "B03001_003E",
                          "B02001_002E",
                          "B17001A_002E",
                          "B17001I_002E",
                          "B17001B_002E"), {'for': 'state:*'})

    # Convert to DataFrame
    census_pd = pd.DataFrame(census_data)

    # Column Reordering
    census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "B19013_001E": "Median Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "B25064_001E": "Median Cost of Rent",
                                      
                                      "B02001_003E":"Black Population",
                                      "B03001_003E":"Hispanic Population",
                                      "B02001_002E":"White Population",
                                      "B17001A_002E":"White Poverty Count",
                                      "B17001I_002E":"Hispanic Poverty Count",
                                      "B17001B_002E":"Black Poverty Count",
                                      "NAME": "Name", "state": "State"})

    # Add in Poverty Rate (Poverty Count / Population)
    census_pd["Poverty Rate"] = 100 * \
        census_pd["Poverty Count"].astype(
            int) / census_pd["Population"].astype(int)

    # Add in Employment Rate (Employment Count / Population)
    census_pd["Unemployment Rate"] = 100 * \
        census_pd["Unemployment Count"].astype(
            int) / census_pd["Population"].astype(int)

    census_pd["Black Poverty Rate"] = 100 * census_pd["Black Poverty Count"].astype(
            int) / census_pd["Black Population"].astype(int)

    census_pd["White Poverty Rate"] = 100 * census_pd["White Poverty Count"].astype(
            int) / census_pd["White Population"].astype(int)

    census_pd["Hispanic Poverty Rate"] = 100 * census_pd["Hispanic Poverty Count"].astype(
            int) / census_pd["Hispanic Population"].astype(int)





    # Final DataFrame
    census_pd = census_pd[["State", "Name", "Population", "Median Household Income",
                       "Per Capita Income", "Poverty Rate", "Unemployment Rate",
                       "Median Cost of Rent", "Black Poverty Rate", "Hispanic Poverty Rate", "White Poverty Rate"]]

    return census_pd

In [32]:
data_2011 = project_data(api_key,2011)
data_2012 = project_data(api_key,2012)
data_2013 = project_data(api_key,2013)
data_2014 = project_data(api_key,2014)
data_2015 = project_data(api_key,2015)
data_2016 = project_data(api_key,2016)
data_2017 = project_data(api_key,2017)

In [33]:
# creating sql connection
connection_string = "postgres:Alevaporo58@localhost:5432/project_2"
engine = create_engine(f'postgresql://{connection_string}')

In [34]:
def to_sql(df_name, table_name):
    
    df_name.to_sql(name = table_name , con=engine, if_exists='append', index=False)

In [35]:
to_sql(data_2011, 'year_2011')
to_sql(data_2012, 'year_2012')
to_sql(data_2013, 'year_2013')
to_sql(data_2014, 'year_2014')
to_sql(data_2015, 'year_2015')
to_sql(data_2016, 'year_2016')
to_sql(data_2017, 'year_2017')

In [36]:
data_2011

,State,Name,Population,Median Household Income,Per Capita Income,Poverty Rate,Unemployment Rate,Median Cost of Rent,Black Poverty Rate,Hispanic Poverty Rate,White Poverty Rate
0,01,Alabama,4747424.0,42934.0,23483.0,17.133186,4.540210,674.0,29.463344,30.578194,12.177266
1,02,Alaska,700703.0,69014.0,31944.0,9.292239,4.414424,1017.0,10.005976,10.009691,6.485448
2,04,Arizona,6337373.0,50752.0,25784.0,15.835820,4.234610,881.0,21.488774,25.156366,13.824388
3,05,Arkansas,2895928.0,40149.0,21833.0,17.846507,3.940015,637.0,32.409194,30.347053,14.371255
4,06,California,36969200.0,61632.0,29634.0,14.096818,5.055211,1185.0,20.357537,20.387118,12.450239
5,08,Colorado,4966061.0,57685.0,30816.0,12.237606,4.093365,883.0,22.856267,23.753170,10.795854
6,09,Connecticut,3558172.0,69243.0,37627.0,9.183676,4.598822,1020.0,19.450424,23.930368,6.580507
7,10,Delaware,890856.0,59317.0,29659.0,10.866515,3.954736,975.0,18.403450,22.287595,8.245192
8,11,District of Columbia,593955.0,61835.0,43993.0,17.144902,5.639316,1135.0,24.943601,13.617627,7.569595
9,12,Florida,18688787.0,47827.0,26733.0,14.336939,5.072978,981.0,24.545293,19.171555,11.894364
